In [1]:
from transformers import pipeline
from transformers import AutoTokenizer
from langchain.schema import Document

c:\Users\Ali Khaled\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
operator torchvision::nms does not exist

In [ ]:
def load_txt_as_document(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    return [Document(page_content=text, metadata={"source": file_path})]


# Summarize Text

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

def chunk_by_tokens(text, max_tokens=512, overlap=50):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    chunks = []
    start = 0
    while start < len(tokens):
        end = start + max_tokens
        chunk = tokens[start:end]
        chunks.append(tokenizer.decode(chunk))
        start += max_tokens - overlap
    return chunks

In [ ]:
summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    tokenizer="facebook/bart-large-cnn"
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Error while downloading from https://huggingface.co/facebook/bart-large-cnn/resolve/main/model.safetensors: HTTPSConnectionPool(host='cas-bridge.xethub.hf.co', port=443): Read timed out.
Trying to resume download...
Device set to use cpu


In [ ]:
docs = load_txt_as_document('try.txt')
chunks = chunk_by_tokens(docs[0].page_content)    

In [ ]:
summaries = []
for chunk in chunks:
    result = summarizer(chunk, max_length=200, min_length=30, do_sample=False)
    summaries.append(result[0]['summary_text'])

Your max_length is set to 200, but your input_length is only 116. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=58)


In [ ]:
final_summary = " ".join(summaries)
print("\nFinal Summary:\n", final_summary)


Final Summary:
 Jörg Schmadtke has been in charge of Alemannia Aachen since 2001. The former Borussia Mönchengladbach goalkeeper has never seen one of his teams win the Bundesliga championship or the DFB Pokal German Cup.


# QA

In [ ]:
qa_model = pipeline(
    "question-answering",
    model="deepset/roberta-base-squad2"
)

c:\Users\Ali Khaled\Desktop\Summarization-QA-Task\.venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ali Khaled\.cache\huggingface\hub\models--deepset--roberta-base-squad2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu


In [ ]:
question = input("\nEnter your question: ")

best_answer = None
best_score = 0
for chunk in chunks:
    result = qa_model(question=question, context=chunk)
    if result['score'] > best_score:
        best_score = result['score']
        best_answer = result['answer']


print(best_answer)

Jörg Schmadtke
